##hash function

In [15]:
import math
import time
import itertools
import random
start = time.time()

#hash function
def hash(element1,element2,num_trans):
  result = (element1 + element2) % num_trans
  return result

##Load Data && generate C1 and pairs for bucket && get random sample of the data with size(1/4)

In [ ]:
candidates = {}
buckets = {}
num_trans = 0
all_data = []
all_data_sample = []
#read all basket from file
with open('retail.txt', 'r') as file:
  for line in file:
    num_trans += 1
    line_list = [int(x) for x in line.strip().split()]
    all_data.append(line_list)
print(len(all_data))

#get random portion of 25% of the file 
all_data_sample = random.sample(all_data, int(num_trans/4))
print(len(all_data_sample))

#count item and hash pairs for sample dataset
def read_data(all_data_sample,num_trans,candidates,buckets):
  for line_list in all_data_sample:  
      for item in line_list:
        if item in candidates.keys():
          candidates.update({item:(candidates[item]+1)})
        else:
          candidates.update({item:1})
      pairs = list(itertools.combinations(line_list, 2))
      for pair in pairs:
        index = hash(pair[0], pair[1],int(num_trans/4))
        if index in buckets.keys():
          buckets.update({index:(buckets[index]+1)})
        else:
          buckets.update({index:1})
  return candidates,buckets
#netflix.data retail.txt

# get c1,buckets from sample dataset
c1,buckets = read_data(all_data_sample,num_trans,candidates,buckets)
sample_size = len(all_data_sample)
min_support = 0.01
#sample size threshold
threshold = int(num_trans*min_support*(1/4))

#original dataset size threshold
true_threshold = int(num_trans*min_support)
#0.01

##Generate Bitmap


In [17]:
#generate bit vector based on the sample size
def generate_Bitmap(hashtable,sample_size,threshold):
  bitmap = [0]*sample_size
  for bucket,bucket_count in hashtable.items():
    if bucket_count >= threshold:
      bitmap[bucket] = 1
  return bitmap

##Generate L1

In [18]:
def get_L(candidates,threshold):
  L = []
  for key,value in candidates.items():
    if value >= threshold:
      L.append(key)
  return L

#get frequent items based on the sample dataset
L1 = get_L(c1,threshold)

##Count initially_pruned C2

In [19]:
def get_C2(candidates,dataset):
  for basket in dataset:
    for key in candidates.keys():
      if set(key).issubset(set(basket)):
        candidates.update({key:candidates[key]+1})
  return candidates

##Generate C2 && bitmap

In [ ]:
#generate C2 based on the frequent items from sample dataset
C2 = list(itertools.combinations(L1, 2))

bitmap = generate_Bitmap(buckets,sample_size,threshold)

print(len(bitmap))

##Generate L2 based on the sample dataset and verified L2 based on the original dataset

In [ ]:
def get_L2(C2,bitmap,all_data_sample,all_data,threshold,true_threshold):
  for pair in C2:
    index = hash(pair[0], pair[1],int(num_trans/4))
    if bitmap[index] != 1:
      C2.remove(pair)
  if len(C2) == 0:
    return None
  else:
    #get L2 from sample dataset
    initial_value = [0]*len(C2)
    C2_initial = dict(zip(C2, initial_value))
    C2_initial = get_C2(C2_initial,all_data_sample)
    L2_sample = get_L(C2_initial,threshold)
    #get verified L2 through sample L2 and original dataset
    sample_value = [0]*len(L2_sample)
    L2_sample_initial = dict(zip(L2_sample, sample_value))
    L2_sample_count = get_C2(L2_sample_initial,all_data)
    L2 = get_L(L2_sample_count,true_threshold)
    return L2_sample,L2

L2_sample,L2 = get_L2(C2,bitmap,all_data_sample,all_data,threshold,true_threshold)

print("pairs containing false positive: ",L2_sample,"\nthe length of it: " , len(L2_sample))

print("pairs after verifying: ",L2,"\nthe length of it: " , len(L2))


print("--- seconds---" , (time.time() - start))